In [ ]:
import telebot
from telebot import types
import websocket
import json
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import io
import talib

In [ ]:
with open('token.txt', 'r') as file:
    TOKEN = file.readline().strip()

# Bot's funcs

ACTIVE_CHATS = []  # List to store active chat IDs

symbol = 'ethusdt'
socket = f'wss://stream.binance.com:9443/ws/{symbol}@kline_1m'

t, o, h, l, c, v = [], [], [], [], [], []
ws = None
df = None
is_streaming = False
previous_df = None


def create_keyboard(start=False,
                    stop=False,
                    get_data=False,
                    graphs=False,
                    return_to_menu=False,
                    plot_price=False,
                    plot_mean=False,
                    plot_variance=False,
                    plot_rsi=False,
                    plot_moving_average=False,
                    plot_stochastic_oscillator=False):
    keyboard = types.ReplyKeyboardMarkup(row_width=2, resize_keyboard=True)

    if start:
        start_button = types.KeyboardButton('/start')
        keyboard.add(start_button)
    if stop:
        stop_button = types.KeyboardButton('/stop')
        keyboard.add(stop_button)
    if get_data:
        get_data_button = types.KeyboardButton('/get_data')
        keyboard.add(get_data_button)
    if graphs:
        graphs_button = types.KeyboardButton('/graphs')
        keyboard.add(graphs_button)
    if return_to_menu:
        return_to_menu_button = types.KeyboardButton('/start_menu')
        keyboard.add(return_to_menu_button)

    if is_streaming:
        if plot_price:
            plot_price_button = types.KeyboardButton('/price')
            keyboard.add(plot_price_button)
        if plot_mean:
            plot_mean_button = types.KeyboardButton('/mean')
            keyboard.add(plot_mean_button)
        if plot_variance:
            plot_variance_button = types.KeyboardButton('/variance')
            keyboard.add(plot_variance_button)
        if plot_rsi:
            plot_rsi_button = types.KeyboardButton('/rsi')
            keyboard.add(plot_rsi_button)
        if plot_moving_average:
            plot_ma_button = types.KeyboardButton('/moving_average')
            keyboard.add(plot_ma_button)
        if plot_stochastic_oscillator:
            plot_so_button = types.KeyboardButton('/stochastic_oscillator')
            keyboard.add(plot_so_button)

    return keyboard





def add_active_chat(chat_id):
    if chat_id not in ACTIVE_CHATS:
        ACTIVE_CHATS.append(chat_id)
    return len(ACTIVE_CHATS)


def remove_active_chat(chat_id):
    if chat_id in ACTIVE_CHATS:
        ACTIVE_CHATS.remove(chat_id)


def start_stream(chat_id):
    global is_streaming, df, ws
    is_streaming = True
    add_active_chat(chat_id)
    message = f"You opened the stream.\n\nStream opened at:\n{start_time}"
    bot.send_message(chat_id, message,
                     reply_markup=create_keyboard(stop=True, get_data=True, graphs=True))
    ws = websocket.WebSocketApp(socket,
                                on_open=on_open,
                                on_message=on_message,
                                on_error=on_error,
                                on_close=on_close)
    ws.run_forever()
    df = None


def stop_stream(chat_id):
    global is_streaming, previous_df, ws
    remove_active_chat(chat_id)
    if len(ACTIVE_CHATS) == 0:
        is_streaming = False
        ws.close()
        previous_df = df
        bot.send_message(chat_id, "You closed the stream.", reply_markup=create_keyboard(start=True))
    else:
        bot.send_message(chat_id, "You have been disconnected from the stream.", reply_markup=create_keyboard(start=True))


def send_data(chat_id):
    global df, previous_df
    if df is not None:
        df.to_csv('data.csv')
        with open('data.csv', 'rb') as file:
            bot.send_document(chat_id, file)
    elif previous_df is not None:
        previous_df.to_csv('data.csv')
        with open('data.csv', 'rb') as file:
            bot.send_document(chat_id, file)
    else:
        bot.send_message(chat_id, "No data to send.", reply_markup=create_keyboard(stop=True, get_data=True, graphs=True))

In [ ]:
# Making graphs

def plot_price(chat_id):
    global df
    if df is not None:
        plt.figure(figsize=(10, 6))  # Increase figure size (width=10, height=6)
        plt.plot(df.index, df['close'], label='Price')
        plt.xticks(rotation=45)
        plt.xlabel('Time')
        plt.ylabel('Price')
        plt.title('ETH Price')
        plt.legend()
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        bot.send_photo(chat_id, buf)
        plt.close()
    else:
        bot.send_message(chat_id, "No available data to plot price.",
                         reply_markup=create_keyboard(stop=True, get_data=True, graphs=True))


def plot_mean(chat_id):
    global df
    if df is not None:
        mean_volume = df['volume'].mean()
        plt.figure(figsize=(10, 6))  # Increase figure size (width=10, height=6)
        plt.plot(df.index, df['volume'], label='Volume')
        plt.xticks(rotation=45)
        plt.axhline(mean_volume, color='r', linestyle='--', label='Mean Volume')
        plt.xlabel('Time')
        plt.ylabel('Volume')
        plt.title('Mean Volume')
        plt.legend()
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        bot.send_photo(chat_id, buf)
        plt.close()
    else:
        bot.send_message(chat_id, "No available data to plot mean volume.",
                         reply_markup=create_keyboard(stop=True, get_data=True, graphs=True))


def plot_variance(chat_id):
    global df
    if df is not None:
        variance = df['volume'].var()
        plt.figure(figsize=(10, 6))  # Increase figure size (width=10, height=6)
        plt.plot(df.index, df['volume'], label='Volume')
        plt.xticks(rotation=45)
        plt.axhline(variance, color='r', linestyle='--', label='Variance')
        plt.xlabel('Time')
        plt.ylabel('Volume')
        plt.title('Variance')
        plt.legend()
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        bot.send_photo(chat_id, buf)
        plt.close()
    else:
        bot.send_message(chat_id, "No available data to plot variance.",
                         reply_markup=create_keyboard(stop=True, get_data=True, graphs=True))


def plot_rsi(chat_id):
    global df
    if df is not None:
        rsi = talib.RSI(df['close'])
        plt.figure(figsize=(10, 6))  # Increase figure size (width=10, height=6)
        plt.plot(df.index, rsi, label='RSI')
        plt.xticks(rotation=45)
        plt.xlabel('Time')
        plt.ylabel('RSI')
        plt.title('Relative Strength Index (RSI)')
        plt.legend()
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        bot.send_photo(chat_id, buf)
        plt.close()
    else:
        bot.send_message(chat_id, "No available data to plot RSI.",
                         reply_markup=create_keyboard(stop=True, get_data=True, graphs=True))


def plot_moving_average(chat_id):
    global df
    if df is not None:
        ma = talib.SMA(df['close'], timeperiod=20)  # Calculate the 20-period simple moving average
        plt.figure(figsize=(10, 6))  # Increase figure size (width=10, height=6)
        plt.plot(df.index, df['close'], label='Price')
        plt.plot(df.index, ma, label='Moving Average')
        plt.xticks(rotation=45)
        plt.xlabel('Time')
        plt.ylabel('Price')
        plt.title('Moving Average')
        plt.legend()
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        bot.send_photo(chat_id, buf)
        plt.close()
    else:
        bot.send_message(chat_id, "No available data to plot stochastic oscillator.",
                         reply_markup=create_keyboard(start=True, get_data=True, graphs=True))


def plot_stochastic_oscillator(chat_id):
    global df
    if df is not None:
        slowk, slowd = talib.STOCH(df['high'], df['low'], df['close'])  # Calculate the stochastic oscillator
        plt.figure(figsize=(10, 6))  # Increase figure size (width=10, height=6)
        plt.plot(df.index, slowk, label='Slow %K')
        plt.plot(df.index, slowd, label='Slow %D')
        plt.xticks(rotation=45)
        plt.xlabel('Time')
        plt.ylabel('Value')
        plt.title('Stochastic Oscillator')
        plt.legend()
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        bot.send_photo(chat_id, buf)
        plt.close()
    else:
        bot.send_message(chat_id, "No available data to plot stochastic oscillator.",
                         reply_markup=create_keyboard(start=True, get_data=True, graphs=True))

In [ ]:
# Websocket funcs

def on_message(ws, message):
    if is_streaming:
        global df
        msg = json.loads(message)
        bar = msg['k']
        if bar['x'] == False:
            timestamp = datetime.datetime.fromtimestamp(bar['t'] / 1000).strftime('%Y-%m-%d %H:%M:%S:%f')
            data = {'time': [timestamp], 'open': [float(bar['o'])], 'high': [float(bar['h'])],
                    'low': [float(bar['l'])], 'close': [float(bar['c'])], 'volume': [float(bar['v'])]}
            new_df = pd.DataFrame(data)
            new_df['time'] = pd.to_datetime(new_df['time'], format='%Y-%m-%d %H:%M:%S:%f')
            new_df.set_index('time', inplace=True)
            if df is None:
                df = new_df
            else:
                df = pd.concat([df, new_df])


def on_error(ws, error):
    print(error)


def on_close(ws, close_status_code, close_msg):
    global is_streaming, df
    if len(ACTIVE_CHATS) == 0:
        is_streaming = False
        print(df)
        df.to_csv('data.csv')


def on_open(ws):
    current_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print(f"Connection opened at {current_time}")
    return current_time

In [ ]:
start_time = on_open(ws)

# Initialize the Telegram bot
bot = telebot.TeleBot(TOKEN)


@bot.message_handler(commands=['start'])
def handle_start(message):
    chat_id = message.chat.id
    if not is_streaming:
        start_stream(chat_id)
    else:
        add_active_chat(chat_id)
        message = f"You are connected to the existing stream.\n\nUsers connected: {len(ACTIVE_CHATS)}\nStream opened at:\n{start_time}"
        bot.send_message(chat_id, message,
                         reply_markup=create_keyboard(stop=True, get_data=True, graphs=True))


@bot.message_handler(commands=['graphs'])
def handle_graphs_menu(message):
    chat_id = message.chat.id
    bot.send_message(chat_id, "Select a graph to plot:",
                     reply_markup=create_keyboard(plot_price=True,
                                                  plot_mean=True,
                                                  plot_variance=True,
                                                  plot_rsi=True,
                                                  plot_moving_average=True,
                                                  plot_stochastic_oscillator=True,
                                                  return_to_menu=True))


@bot.message_handler(commands=['start_menu'])
def handle_start_menu(message):
    chat_id = message.chat.id
    bot.send_message(chat_id, "Main Menu:",
                     reply_markup=create_keyboard(stop=True, get_data=True, graphs=True))

@bot.message_handler(commands=['stop'])
def handle_stop(message):
    chat_id = message.chat.id
    stop_stream(chat_id)


@bot.message_handler(commands=['get_data'])
def handle_get_data(message):
    chat_id = message.chat.id
    send_data(chat_id)


@bot.message_handler(commands=['price'])
def handle_plot_price(message):
    chat_id = message.chat.id
    plot_price(chat_id)


@bot.message_handler(commands=['mean'])
def handle_plot_mean(message):
    chat_id = message.chat.id
    plot_mean(chat_id)


@bot.message_handler(commands=['variance'])
def handle_plot_variance(message):
    chat_id = message.chat.id
    plot_variance(chat_id)


@bot.message_handler(commands=['rsi'])
def handle_plot_rsi(message):
    chat_id = message.chat.id
    plot_rsi(chat_id)


@bot.message_handler(commands=['moving_average'])
def handle_plot_moving_average(message):
    chat_id = message.chat.id
    plot_moving_average(chat_id)


@bot.message_handler(commands=['stochastic_oscillator'])
def handle_plot_stochastic_oscillator(message):
    chat_id = message.chat.id
    plot_stochastic_oscillator(chat_id)


@bot.message_handler(func=lambda message: True)
def handle_invalid(message):
    chat_id = message.chat.id
    bot.send_message(chat_id, "Invalid command. Please use the available commands.",
                     reply_markup=create_keyboard(start=True))


bot.polling()